In [1]:
import pandas as pd
from datetime import datetime, timedelta
version_in = 'V0_1'
version_out = 'V1_4'
path_gen = "/home/pol/Escritorio/INVERBIS/2024/CancerColon/"
path_1 = path_gen + "Data/processed_data/CancerColon_filtered"+str(version_out)+".csv"
df = pd.read_csv(path_1, sep= ';')

In [2]:
print(df.shape)
print(df.columns)
df['Actividad'].unique().tolist()

(4515, 62)
Index(['NASI seudonimizado', 'Subg. químico terapéutico ATC disp',
       'Principio activo ATC disp', 'Prezo PVP nomen disp', 'FECHA',
       'FECHA_FIN', 'Actividad', 'Hospital', 'Motivo de asistencia',
       'Motivo saída', 'PAC', 'Lugar asistencia', 'Motivo asistencia', 'Acto',
       'Prioridade', 'GNA', 'Actividade CAPNOR', 'Código Prestación',
       'Prestación', 'Tipo vía rápida', 'Detalle tipo actividade',
       'Tipo programación', 'Cola', 'Tipo intervención',
       'Orixe PM/Diagnóstico', 'PM/Diagnóstico', 'Categoría CIE',
       'Subcategoría CIE', 'CENTRO', 'SERVICIO', 'MEDICO', 'MUESTRA',
       'DESCRIPCION', 'DATA EXTRACCIÓN', 'NUM_LAXANTES', 'NUM_QUIMIOS',
       'Defuncion', 'VDR', 'Colonoscopia',
       'Consultas Digestivo/Medicina interna',
       'Consultas Cirugía General/Oncología', 'Realización TC abdominales',
       'Duracion Lista Espera', 'Medicamento importante',
       'Nº de compras farmaceuticas', 'Coste total medicamentos',
       'num_c

['Cita en Medicina interna',
 'Biopsia diagnóstica',
 'Ingreso hospitalario',
 'Cirugia Programada',
 'Alta Hospitalizacion',
 'Visita médico familia',
 'Defunción',
 'Cita en Cirugía General',
 'Quimioterapia',
 'Cita en Digestivo',
 'Cirugia Urgente',
 'Cita en Oncología']

In [3]:
#! crear forma de contar tiempos entre actividades no consecutivas
# - Última consulta en Medicina de familia y biopsia.
# - Biopsia y cirugia (percentiles de duraciones)
# - Biopsia y TC abdominal.
# - Biopsia y consulta de Oncología.
# - Biopsia y consulta de Cirugía.
# - Cirugía y Quimio.
# - Quimio y siguiente atención en Urgencias.

def duration_no_consecutive_arcs(df, activ_list, criteria = 1):
    group = df.groupby('NASI seudonimizado')
    df['FECHA'] = pd.to_datetime(df['FECHA'], format= '%d-%m-%Y %H:%M:%S')
    df['FECHA_FIN'] = pd.to_datetime(df['FECHA_FIN'], format= '%d-%m-%Y %H:%M:%S')
    names = ['NASI seudonimizado']
    traces = []
    for name, trace in group:
        for arc in activ_list:
            activ_1 = arc[0]
            activ_2 = arc[-1]
            trace[f'Duracion_{activ_1}_{activ_2}'] = 0
            names.append(f'Duracion_{activ_1}_{activ_2}')
            #buscar primewra y ultima apariciṕn de la primera y segunda actividad
            if activ_1 in trace['Actividad'].values and activ_2 in trace['Actividad'].values:
                # Primera aparición de primera actividad
                primer_biopsia = trace[trace['Actividad'] == activ_1]['FECHA'].idxmin()
                fecha_primer_primera = df.loc[primer_biopsia, 'FECHA']
                # Última aparición de primera actividad
                ultima_biopsia = trace[trace['Actividad'] == activ_1]['FECHA'].idxmax()
                fecha_ultima_primera = trace.loc[ultima_biopsia, 'FECHA']

                # Primera aparición de segunda actividad
                primer_cita = trace[trace['Actividad'] == activ_2]['FECHA_FIN'].idxmin()
                fecha_primer_segunda = trace.loc[primer_cita, 'FECHA_FIN']
                # Última aparición de segunda actividad
                ultima_cita = trace[trace['Actividad'] == activ_2]['FECHA_FIN'].idxmax()
                fecha_ultima_segunda = trace.loc[ultima_cita, 'FECHA_FIN']
                # print(f"Primera aparición de {activ_1}:", fecha_primer_primera)
                # print(f"Última aparición de {activ_1}:", fecha_ultima_primera)
                # print(f"Primera aparición de {activ_2}:", fecha_primer_segunda)
                # print(f"Última aparición de {activ_2}:", fecha_ultima_segunda)

                #restar fecha_fin segunda actividad menos fecha_inicio primera actividad siguiendo uno de los cuatro criterios:
                if criteria == 1:
                    diferencia_segundos = (fecha_primer_segunda - fecha_primer_primera).total_seconds()
                elif criteria == 2:
                    diferencia_segundos = (fecha_ultima_segunda - fecha_primer_primera).total_seconds()
                elif criteria == 3:
                    diferencia_segundos = (fecha_ultima_segunda - fecha_ultima_primera).total_seconds()
                elif criteria == 4:
                    diferencia_segundos = (fecha_primer_segunda - fecha_ultima_primera).total_seconds()
                else:
                    print('Esta opcion no es valida')

                if diferencia_segundos >=0:
                    print(diferencia_segundos)
                    trace[f'Duracion_{activ_1}_{activ_2}'] = diferencia_segundos
            traces.append(trace)
    cross_df_raw = pd.concat(traces)
    cross_df_raw = cross_df_raw[names]
    cross_df_raw = cross_df_raw.drop_duplicates(subset=['NASI seudonimizado'], keep='first')
    return cross_df_raw.iloc[:, :len(arc_list) + 1]


arc_list = [['Colonoscopia diagnóstica', 'Cita en Oncología'], ['Colonoscopia diagnóstica', 'Cirugia Programada'], ['Colonoscopia diagnóstica', 'Cirugia Urgente']]

cross_df  = duration_no_consecutive_arcs(df, arc_list)


2372704.0
2199766.0
5053513.0
3670398.0
4188798.0
816815.0
1248596.0
1248842.0
1766155.0
41942155.0
1855440.0
1681629.0
3235576.0
126796.0
41427.0
299098.0
3063506.0
3410784.0
1248139.0
556246.0
13257046.0
2895858.0
1179692.0
2459749.0
1076252.0
1940252.0
1077184.0
12827584.0
1076904.0
4703308.0
5221708.0
1251020.0
13347020.0
472827.0
2628603.0
2974203.0
3065239.0
1333768.0
2195530.0
2804879.0
1336249.0
12049849.0
1854748.0
1249657.0
730765.0
2456348.0
1767592.0
2979672.0
1697498.0
2373024.0
3498422.0
4183761.0
13342161.0
3672896.0
4191296.0
2720031.0
5485848.0
1681487.0
1941509.0
1421263.0
3495992.0
38055992.0
1934508.0
2628406.0
2459569.0
1852673.0
17923073.0
1946074.0
1940063.0
817490.0
1077091.0
8075491.0
1853476.0
2802618.0
2284787.0
558397.0
1768421.0
1163279.0
1769188.0
2457583.0
127252.0
816779.0
10407179.0
3670406.0
2197563.0
2196658.0
3406258.0
2113605.0
1938961.0
2887031.0
989510.0
5052062.0
2803241.0
1077237.0
1336025.0
6950263.0
644500.0
1076286.0
985760.0
2458591.0
254369

In [5]:
cross_df.head(20)

,NASI seudonimizado,Duracion_Biopsia diagnóstica_Cita en Oncología,Duracion_Biopsia diagnóstica_Cirugia Programada,Duracion_Biopsia diagnóstica_Cirugia Urgente
0,0067638e545bc471e1f89de1738c1d5c,0.0,2372704.0,0.0
5,010b7970230a0be9031b74e8aff8df0f,0.0,0.0,0.0
9,0129d3e82128fa38a023dc2b8d2bf445,0.0,2199766.0,0.0
15,017f4d4811a20947be153d6f8f49e1bd,0.0,5053513.0,0.0
22,01ca2f0d170fd6ecf91984415350ce2d,0.0,3670398.0,4188798.0
36,03cd9c70a41eedebc2c766105812f625,816815.0,0.0,0.0
44,048f62dc5a6ee6f08bf556c8c83efd2f,1248596.0,0.0,0.0
64,04eab71184ffc43b74b8363cc8e20b21,1248842.0,0.0,0.0
68,060222112f2f2d7de70a5fee03587584,0.0,1766155.0,41942155.0
77,0675a25702af527a638e198c682d6666,0.0,1855440.0,0.0
